In [91]:
import sys
sys.path.append("chinna-grad")
from engine import Node


SyntaxError: invalid syntax (1341715216.py, line 1)

In [88]:

# Visualize the connections
from graphviz import Digraph as dir_graph

def trace(result):
    # Use DFS to build a set of all nodes and edges starting from the final result node
    nodes = set()
    edges = set()
    def build(node):
        if (node not in nodes):
            nodes.add(node)
            for child in (node._prev):
                edges.add((child, node))
                build(child)
    build(result)
    return (nodes, edges)

def draw_graph(result):
    graph = dir_graph(format = 'svg', graph_attr = {'rankdir': "LR"})  # Draw the graph from left to right
    nodes, edges = trace(result)
    for node in nodes:
        # For every node, create a rectangle
        graph.node(name = node.name, label = "{%s | data: %.3f | gradient: %.3f}" % (node.name, node.data, node.grad), shape = "record")

        # For non-input nodes, create an operation node behind it
        if node._op:
            opNodeName = node.name + node._op
            graph.node(name = opNodeName, label = node._op)
            graph.edge(opNodeName, node.name)
    
    for node1, node2 in edges:
        graph.edge(node1.name, node2.name + node2._op)
    
    return graph

In [89]:
a = Node(3.0)
b = Node(-3.0)
c = a * b
d = Node(10.0)
e  = c  + d


#-------Trying out tanh implementation from scratch-----
# f = (2*e).exp()
# O = (f - 1) / (f + 1)
# O.name = "O"
# O.backward()
# draw_graph(O)

O = e.tanh()
O.name = "O"
O.backward()
draw_graph(O)



NameError: name 'math' is not defined